In [123]:
import numpy as np
from numpy import linspace, loadtxt, ones, convolve
import pandas as pd
#import pickle
#from textblob import TextBlob
#import matplotlib.pyplot as plt
#import matplotlib.lines as ln
#from matplotlib.dates import YearLocator, MonthLocator, DateFormatter, drange , date2num
import json
import os
import nltk
from nltk import RegexpTokenizer
#from collections import Counter
import re
from datetime import datetime
import time
#from scipy.signal import savgol_filter
import string

from gensim import matutils, models
import scipy.sparse

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [124]:
global_path = '~/Desktop/Datasets/art/art_writing'
path = 'artnet_articles'
#path = 'artnet_articles'
#path = 'artnet_articles'
folderpath = os.path.expanduser(os.path.join(global_path, path))

filelist = os.listdir(folderpath)

In [126]:
data = {}
data = pd.DataFrame(data)

cleaned_titles = []
cleaned_paras = []
brought_in = 0

for file in filelist: #[:100]:
    #file = filelist.sample(1000)
    filepath = os.path.join(folderpath, file)
    f = open(filepath) #, encoding='ascii', errors='ignore')
    try:
        j_import = json.load(f)
        j_import = j_import[0]
        #print(j_import)
        #title = j_import['title']
        try:
            para = j_import['para']
            para = para[0]
            tokenizer = RegexpTokenizer(r'\w+')
            tokens = tokenizer.tokenize(para)
            #print(tokens)
            count = len(tokens)
            if count > 1000 and brought_in < 300:
                tokens =[]
                brought_in += 1
                
                para = para.replace("	", "").replace("Follow  on Facebook:", '').replace("\n", ' ')
                para = para.replace("\r" , " ")
                para = para.strip()
                #print(para)

                title = j_import['title']
                title = title.replace("	", "").replace("\n", ' ')
                title = title.replace("\r" , " ")
                title = title.strip()

                author = j_import['author']
                author = author.replace("	", "").replace("\n", ' ')
                author = author.replace("\r" , " ")
                author = author.strip()

                pubtime_i = j_import['pubtime']
                pubtime_i = pubtime_i[:19]

                pubtime = datetime(*time.strptime(pubtime_i, "%Y-%m-%dT%H:%M:%S")[:6])

                new_row = {"title" : title , "para" : para , "author": author ,
                           "pubtime" : pubtime }
                data = data.append(new_row, ignore_index=True)
                f.close()
            else:
                pass

        except KeyError:
            f.close()
            pass

    except json.decoder.JSONDecodeError:
        f.close()
        pass

#data = data.set_index('title')
data

,author,para,pubtime,title
0,Artnet News,"Art Industry News is a daily digest of the most consequential developments coming out of the art world and art market. Here’s what you need to know this Wednesday, March 20. NEED-TO-READ The Minimalist painter has had an unusually public and candid struggle with cancer, but his illness has not changed his artistic practice. “It hasn’t made me work any differently,” Marden says. “It’s just been an extra thing to think about.” The 80-year-old artist, who is based in upstate New York, is showin...",2019-03-20 13:16:19,Art Industry News: Obama Portraitist Kehinde Wiley Is Launching His Own Artist Residency in Senegal + Other Stories
1,Caroline Goldstein,"In another installment of our series investigating the final paintings of famous artists , the list takes a darker turn, with many of the artists aware that death was near. That knowledge manifested itself differently, from Stuart Davis’s poetic resignation to the revelation of Marcel Duchamp’s closely kept secret. Once again we used Bernard Chambaz’s book The Last Painting as a jumping off point while adding our own research and insight. Below, see a selection of the works created by artist...",2019-11-08 17:18:05,"See the Poetic, Heartbreaking Final Paintings of 7 Famous Artists, From Salvador Dalí to Marcel Duchamp"
2,Brian Boucher,"As museum acquisitions grow, and amid a ballooning number of museums around the world, New York’s New Museum has mounted a show centered around the practice of collecting. The museum itself doesn’t collect, so the show may come with a dose of irony; the curators may intend, with this sprawling show, a knowing nod to the museum enterprise itself. Might the acquisitive urges of business-suited boards of trustees be so distinct from those of the hoarder in apartment 2B? But “The Keeper” isn’t j...",2016-08-12 15:21:58,"From Auschwitz Drawings to Teddy Bears, ‘The Keeper’ Studies the Desire to Preserve"
3,Vivienne Chow,"On June 30, the eve of the 23rd anniversary of Britain’s handover of Hong Kong to China, Beijing passed a national security law that changed the semi-autonomous city overnight. The law, which bypassed the local legislature and prohibits acts and activities related to secession, subversion of the state, terrorism and collusion with foreign forces, gives police sweeping powers to conduct warrant-less searches and surveillance. Certain protest slogans and songs have been deemed illegal. Books b...",2020-07-16 12:59:43,Beijing Is Clamping Down on ‘Subversive’ Activity in Hong Kong. The Art Market Doesn’t Seem to Care
4,Taylor Dafoe,"A year ago, artist Allison Zuckerman was 26, fresh out of grad school, and frustrated. She was working for a gallery in SoHo and painting in her free time, but she wasn’t showing anywhere. So she began posting her work to Instagram instead. “I would recycle old work—things that I had done in grad school or college—and then Photoshop it, turn it into new compositions,” Zuckerman recalls. “I felt like, ‘What am I going to do, carry my work through the galleries in Chelsea and say please look?’...",2017-12-05 14:14:32,Behind Artist Allison Zuckerman’s Rapid Rise From Gallery Assistant to the Rubell Family’s Newest Obsession
...,...,...,...,...
295,Sarah Cascone,"On November 15, Wayne Thiebaud , the cherished American painter of cakes and sweets, hits an impressive milestone: his 100th birthday. Born November 1920 in Mesa, Arizona, and raised in California, Thiebaud remains busy and productive even on the eve of his centenary. “I spend a lot of time in the studio,” he told Artnet News. “I work almost every day. I have a wonderful family who indulges me in this amazing search for how difficult it is to make a good painting.” Thiebaud’s birthday is bei...",2020-11-13 22:38:06,"‘Enjoy It When You Have It, But Don’t Have Too Much’: Artist Wayne Thiebaud on How to Savor Cake While Staying Healthy at 100 Years Old"
296,Jasmin Hernandez,"MECA international art fair persists no matte

In [127]:
data = data.set_index('title')
data

,author,para,pubtime
title,,,
Art Industry News: Obama Portraitist Kehinde Wiley Is Launching His Own Artist Residency in Senegal + Other Stories,Artnet News,"Art Industry News is a daily digest of the most consequential developments coming out of the art world and art market. Here’s what you need to know this Wednesday, March 20. NEED-TO-READ The Minimalist painter has had an unusually public and candid struggle with cancer, but his illness has not changed his artistic practice. “It hasn’t made me work any differently,” Marden says. “It’s just been an extra thing to think about.” The 80-year-old artist, who is based in upstate New York, is showin...",2019-03-20 13:16:19
"See the Poetic, Heartbreaking Final Paintings of 7 Famous Artists, From Salvador Dalí to Marcel Duchamp",Caroline Goldstein,"In another installment of our series investigating the final paintings of famous artists , the list takes a darker turn, with many of the artists aware that death was near. That knowledge manifested itself differently, from Stuart Davis’s poetic resignation to the revelation of Marcel Duchamp’s closely kept secret. Once again we used Bernard Chambaz’s book The Last Painting as a jumping off point while adding our own research and insight. Below, see a selection of the works created by artist...",2019-11-08 17:18:05
"From Auschwitz Drawings to Teddy Bears, ‘The Keeper’ Studies the Desire to Preserve",Brian Boucher,"As museum acquisitions grow, and amid a ballooning number of museums around the world, New York’s New Museum has mounted a show centered around the practice of collecting. The museum itself doesn’t collect, so the show may come with a dose of irony; the curators may intend, with this sprawling show, a knowing nod to the museum enterprise itself. Might the acquisitive urges of business-suited boards of trustees be so distinct from those of the hoarder in apartment 2B? But “The Keeper” isn’t j...",2016-08-12 15:21:58
Beijing Is Clamping Down on ‘Subversive’ Activity in Hong Kong. The Art Market Doesn’t Seem to Care,Vivienne Chow,"On June 30, the eve of the 23rd anniversary of Britain’s handover of Hong Kong to China, Beijing passed a national security law that changed the semi-autonomous city overnight. The law, which bypassed the local legislature and prohibits acts and activities related to secession, subversion of the state, terrorism and collusion with foreign forces, gives police sweeping powers to conduct warrant-less searches and surveillance. Certain protest slogans and songs have been deemed illegal. Books b...",2020-07-16 12:59:43
Behind Artist Allison Zuckerman’s Rapid Rise From Gallery Assistant to the Rubell Family’s Newest Obsession,Taylor Dafoe,"A year ago, artist Allison Zuckerman was 26, fresh out of grad school, and frustrated. She was working for a gallery in SoHo and painting in her free time, but she wasn’t showing anywhere. So she began posting her work to Instagram instead. “I would recycle old work—things that I had done in grad school or college—and then Photoshop it, turn it into new compositions,” Zuckerman recalls. “I felt like, ‘What am I going to do, carry my work through the galleries in Chelsea and say please look?’...",2017-12-05 14:14:32
...,...,...,...
"‘Enjoy It When You Have It, But Don’t Have Too Much’: Artist Wayne Thiebaud on How to Savor Cake While Staying Healthy at 100 Years Old",Sarah Cascone,"On November 15, Wayne Thiebaud , the cherished American painter of cakes and sweets, hits an impressive milestone: his 100th birthday. Born November 1920 in Mesa, Arizona, and raised in California, Thiebaud remains busy and productive even on the eve of his centenary. “I spend a lot of time in the studio,” he told Artnet News. “I work almost every day. I have a wonderful family who indulges me in this amazing search for how difficult it is to make a good painting.” Thiebaud’s birthday is bei...",2020-11-13 22:38:06
"Back After the Hurricane, Puerto Rico’s MECA Art Fair Is Making Its Mark Through 

In [128]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = text.replace('-', ' ')
    text = text.replace("’s ", " ").replace("' ", " ")
    text = text.replace("s’ ", " ").replace("s' ", " ")#remove proper possesives
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('new york', 'newyork', text)
    text = re.sub('newyork city', 'newyorkcity', text)
    text = re.sub('new year', 'newyear', text)
    text = re.sub('the new museum', 'newmuseum', text)
    #text = text.replace(r'[^\u0020-\u007E]', '')
    return text

round2 = lambda x: clean_text_round2(x)

# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

<>:7: DeprecationWarning: invalid escape sequence \[
<>:9: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \[
<>:9: DeprecationWarning: invalid escape sequence \w
<ipython-input-128-bc64b8a14122>:7: DeprecationWarning: invalid escape sequence \[
  text = re.sub('\[.*?\]', '', text)
<ipython-input-128-bc64b8a14122>:9: DeprecationWarning: invalid escape sequence \w
  text = re.sub('\w*\d\w*', '', text)


In [129]:
data_clean = pd.DataFrame(data.para.apply(round1))
pd.set_option('display.max_colwidth',500)
data_clean

,para
title,
Art Industry News: Obama Portraitist Kehinde Wiley Is Launching His Own Artist Residency in Senegal + Other Stories,art industry news is a daily digest of the most consequential developments coming out of the art world and art market here what you need to know this wednesday march need to read the minimalist painter has had an unusually public and candid struggle with cancer but his illness has not changed his artistic practice “it hasn’t made me work any differently” marden says “it just been an extra thing to think about” the year old artist who is based in upstate new york is showing little sign of s...
"See the Poetic, Heartbreaking Final Paintings of 7 Famous Artists, From Salvador Dalí to Marcel Duchamp",in another installment of our series investigating the final paintings of famous artists the list takes a darker turn with many of the artists aware that death was near that knowledge manifested itself differently from stuart davis poetic resignation to the revelation of marcel duchamp closely kept secret once again we used bernard chambaz book the last painting as a jumping off point while adding our own research and insight below see a selection of the works created by artists just before...
"From Auschwitz Drawings to Teddy Bears, ‘The Keeper’ Studies the Desire to Preserve",as museum acquisitions grow and amid a ballooning number of museums around the world new york new museum has mounted a show centered around the practice of collecting the museum itself doesn’t collect so the show may come with a dose of irony the curators may intend with this sprawling show a knowing nod to the museum enterprise itself might the acquisitive urges of business suited boards of trustees be so distinct from those of the hoarder in apartment but “the keeper” isn’t just about col...
Beijing Is Clamping Down on ‘Subversive’ Activity in Hong Kong. The Art Market Doesn’t Seem to Care,on june the eve of the anniversary of britain handover of hong kong to china beijing passed a national security law that changed the semi autonomous city overnight the law which bypassed the local legislature and prohibits acts and activities related to secession subversion of the state terrorism and collusion with foreign forces gives police sweeping powers to conduct warrant less searches and surveillance certain protest slogans and songs have been deemed illegal books by pro democracy a...
Behind Artist Allison Zuckerman’s Rapid Rise From Gallery Assistant to the Rubell Family’s Newest Obsession,a year ago artist allison zuckerman was fresh out of grad school and frustrated she was working for a gallery in soho and painting in her free time but she wasn’t showing anywhere so she began posting her work to instagram instead “i would recycle old work—things that i had done in grad school or college—and then photoshop it turn it into new compositions” zuckerman recalls “i felt like ‘what am i going to do carry my work through the galleries in chelsea and say please look’ the only way t...
...,...
"‘Enjoy It When You Have It, But Don’t Have Too Much’: Artist Wayne Thiebaud on How to Savor Cake While Staying Healthy at 100 Years Old",on november wayne thiebaud the cherished american painter of cakes and sweets hits an impressive milestone his birthday born november in mesa arizona and raised in california thiebaud remains busy and productive even on the eve of his centenary “i spend a lot of time in the studio” he told artnet news “i work almost every day i have a wonderful family who indulges me in this amazing search for how difficult it is to make a good painting” thiebaud birthday is being celebrated in a big way...
"Back After the Hurricane, Puerto Rico’s MECA Art Fair Is Making Its Mark Through Pure Enthusiasm",meca international art fair persists no matter what you throw at it the budding contemporary art fair launched last june one month after puerto rico declared a form of bankruptcy it returned this month november in old san juan fourteen m

In [130]:
"""raw text"""
data_clean = pd.DataFrame(data_clean.para.apply(round2))
data_clean

,para
title,
Art Industry News: Obama Portraitist Kehinde Wiley Is Launching His Own Artist Residency in Senegal + Other Stories,art industry news is a daily digest of the most consequential developments coming out of the art world and art market here what you need to know this wednesday march need to read the minimalist painter has had an unusually public and candid struggle with cancer but his illness has not changed his artistic practice it hasnt made me work any differently marden says it just been an extra thing to think about the year old artist who is based in upstate newyork is showing little sign of slowing...
"See the Poetic, Heartbreaking Final Paintings of 7 Famous Artists, From Salvador Dalí to Marcel Duchamp",in another installment of our series investigating the final paintings of famous artists the list takes a darker turn with many of the artists aware that death was near that knowledge manifested itself differently from stuart davis poetic resignation to the revelation of marcel duchamp closely kept secret once again we used bernard chambaz book the last painting as a jumping off point while adding our own research and insight below see a selection of the works created by artists just before...
"From Auschwitz Drawings to Teddy Bears, ‘The Keeper’ Studies the Desire to Preserve",as museum acquisitions grow and amid a ballooning number of museums around the world newyork new museum has mounted a show centered around the practice of collecting the museum itself doesnt collect so the show may come with a dose of irony the curators may intend with this sprawling show a knowing nod to the museum enterprise itself might the acquisitive urges of business suited boards of trustees be so distinct from those of the hoarder in apartment but the keeper isnt just about collecti...
Beijing Is Clamping Down on ‘Subversive’ Activity in Hong Kong. The Art Market Doesn’t Seem to Care,on june the eve of the anniversary of britain handover of hong kong to china beijing passed a national security law that changed the semi autonomous city overnight the law which bypassed the local legislature and prohibits acts and activities related to secession subversion of the state terrorism and collusion with foreign forces gives police sweeping powers to conduct warrant less searches and surveillance certain protest slogans and songs have been deemed illegal books by pro democracy a...
Behind Artist Allison Zuckerman’s Rapid Rise From Gallery Assistant to the Rubell Family’s Newest Obsession,a year ago artist allison zuckerman was fresh out of grad school and frustrated she was working for a gallery in soho and painting in her free time but she wasnt showing anywhere so she began posting her work to instagram instead i would recycle old work—things that i had done in grad school or college—and then photoshop it turn it into new compositions zuckerman recalls i felt like what am i going to do carry my work through the galleries in chelsea and say please look the only way that pe...
...,...
"‘Enjoy It When You Have It, But Don’t Have Too Much’: Artist Wayne Thiebaud on How to Savor Cake While Staying Healthy at 100 Years Old",on november wayne thiebaud the cherished american painter of cakes and sweets hits an impressive milestone his birthday born november in mesa arizona and raised in california thiebaud remains busy and productive even on the eve of his centenary i spend a lot of time in the studio he told artnet news i work almost every day i have a wonderful family who indulges me in this amazing search for how difficult it is to make a good painting thiebaud birthday is being celebrated in a big way in ...
"Back After the Hurricane, Puerto Rico’s MECA Art Fair Is Making Its Mark Through Pure Enthusiasm",meca international art fair persists no matter what you throw at it the budding contemporary art fair launched last june one month after puerto rico declared a form of bankruptcy it returned this month november in old san juan fourteen m

In [137]:
from gensim.utils import simple_preprocess

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_clean, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_clean], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

2021-03-14 20:17:24,750 : INFO : collecting all words and their counts
2021-03-14 20:17:24,752 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2021-03-14 20:17:24,754 : INFO : collected 6 word types from a corpus of 4 words (unigram + bigrams) and 1 sentences
2021-03-14 20:17:24,755 : INFO : using 6 counts as vocab in Phrases<0 vocab, min_count=5, threshold=100, max_vocab_size=40000000>
2021-03-14 20:17:24,757 : INFO : collecting all words and their counts
2021-03-14 20:17:24,758 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2021-03-14 20:17:24,759 : INFO : collected 6 word types from a corpus of 4 words (unigram + bigrams) and 1 sentences
2021-03-14 20:17:24,760 : INFO : using 6 counts as vocab in Phrases<0 vocab, min_count=5, threshold=100, max_vocab_size=40000000>
2021-03-14 20:17:24,761 : INFO : source_vocab length 6
2021-03-14 20:17:24,764 : INFO : Phraser built with 0 phrasegrams
2021-03-14 20:17:24,765 : INFO : source_vocab length 6


In [157]:
# Apply a third round of cleaning
def clean_text_round3(text):
    '''build and apply gensim bigram trigram'''
    text = make_bigrams(text)
    text = make_trigrams(text)
    text = lemmatization(text)
    text = re.sub('[‘’“”…]', '', text)
    #text = text.replace(r'[^\u0020-\u007E]', '')
    return text

round3 =  clean_text_round3

In [160]:
"""raw text"""
data_clean = pd.DataFrame(data_clean.para.apply(round3))
data_clean

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [162]:
#!/usr/bin/python3 -m pip install spacy
import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        text_out = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
    return texts_out


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [136]:
import spacy# Remove Stop Words
#data_clean_nostops = remove_stopwords(data_clean)# Form Bigrams

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
data_clean_bigrams = make_bigrams(data_clean)

# Do lemmatization keeping only noun, adj, vb, adv
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

data_clean_lemmatized = lemmatization(data_clean_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_clean_lemmatized[:1])

NameError: name 'bigram_mod' is not defined

In [43]:
"""one or the other"""
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.para.apply(nouns))
data_nouns = data_clean

In [139]:
"""one or the other"""
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.para.apply(nouns_adj))
data_nouns_adj = data_clean

In [140]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

add_stop_words = ['art', 'artist', 'artists', 'work' , "like"]

# Add new stop words
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate document-term matrix
cv = CountVectorizer(stop_words=stop_words) #"english")
data_cv = cv.fit_transform(data_clean.para)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index

# One of the required inputs is a term-document matrix
tdm = data_dtm.transpose()
tdm.head()

title,Art Industry News: Obama Portraitist Kehinde Wiley Is Launching His Own Artist Residency in Senegal + Other Stories,"See the Poetic, Heartbreaking Final Paintings of 7 Famous Artists, From Salvador Dalí to Marcel Duchamp","From Auschwitz Drawings to Teddy Bears, ‘The Keeper’ Studies the Desire to Preserve",Beijing Is Clamping Down on ‘Subversive’ Activity in Hong Kong. The Art Market Doesn’t Seem to Care,Behind Artist Allison Zuckerman’s Rapid Rise From Gallery Assistant to the Rubell Family’s Newest Obsession,Strong Sales in London for Italian Postwar Market,Experts Weigh In on Jeff Koons Copyright Infringement Lawsuit,Picasso and Modigliani Lead Sotheby’s $151 Million Impressionist and Modern Sale in London,Lee Krasner Had to Fight Her Way Into the History Books. Here are 5 of the Biggest Battles She Faced,Museums Have Long Relied on the Money They Raise at Glitzy Galas. What Can They Do Now That the Party’s Over?,...,Looking to Turn a New Page? Here Are 17 Incredible Art Books to Read That Will Open Up New Horizons for Your Mind Under Lockdown,18 Things You Should See During New York City’s Print Week 2017,"‘In the Art World, Revolution Is in the Air’: Artist Ed Ruscha and Music Legend Jimmy Iovine on How Art Guides Political Change","How a New Kind of Artist Contract Could Provide a Simple, Effective Way to Redistribute the Art Market’s Wealth",Art Industry News: Are Auction Houses Burning Out Their Biggest Stars? + Other Stories,"‘Enjoy It When You Have It, But Don’t Have Too Much’: Artist Wayne Thiebaud on How to Savor Cake While Staying Healthy at 100 Years Old","Back After the Hurricane, Puerto Rico’s MECA Art Fair Is Making Its Mark Through Pure Enthusiasm","Interview with Philip Tinari, Director of the Ullens Center for Contemporary Art","With a ‘Car Crash’ of Biennials Now Planned for 2021, Curators Are Being Forced to Rethink Mass Art Shows for the Social-Distancing Era","The Carnegie International Wants You to Feel ‘Museum Joy’ Again, With Mr. Rogers, Vietnamese Coffee, and Its Own Past"
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aadla,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aam,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aamd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aanant,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
#data_dtm, data_clean = [] , []

In [141]:
#put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [142]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
#cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
#id2word

In [143]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=10, random_state=100,
                      chunksize=100, passes=50, per_word_topics=True)
lda.print_topics()

2021-03-14 20:31:07,944 : INFO : using symmetric alpha at 0.1
2021-03-14 20:31:07,946 : INFO : using symmetric eta at 0.1
2021-03-14 20:31:07,955 : INFO : using serial LDA version on this node
2021-03-14 20:31:08,001 : INFO : running online (multi-pass) LDA training, 10 topics, 50 passes over the supplied corpus of 300 documents, updating model once every 100 documents, evaluating perplexity every 300 documents, iterating 50x with a convergence threshold of 0.001000
2021-03-14 20:31:08,031 : INFO : PROGRESS: pass 0, at document #100/300
2021-03-14 20:31:08,288 : INFO : merging changes from 100 documents into a model of 300 documents
2021-03-14 20:31:08,322 : INFO : topic #9 (0.100): 0.007*"museum" + 0.005*"gallery" + 0.004*"newyork" + 0.004*"pm" + 0.004*"time" + 0.004*"works" + 0.003*"fair" + 0.003*"world" + 0.003*"new" + 0.002*"view"
2021-03-14 20:31:08,330 : INFO : topic #5 (0.100): 0.006*"works" + 0.006*"time" + 0.005*"pm" + 0.004*"museum" + 0.004*"million" + 0.004*"fair" + 0.003*"w

2021-03-14 20:31:12,140 : INFO : topic #8 (0.100): 0.007*"vermeer" + 0.003*"ca" + 0.003*"born" + 0.003*"ny" + 0.003*"lives" + 0.002*"drahi" + 0.002*"time" + 0.002*"ant" + 0.001*"girl" + 0.001*"museum"
2021-03-14 20:31:12,142 : INFO : topic diff=0.436273, rho=0.447214
2021-03-14 20:31:12,179 : INFO : PROGRESS: pass 2, at document #100/300
2021-03-14 20:31:12,478 : INFO : merging changes from 100 documents into a model of 300 documents
2021-03-14 20:31:12,504 : INFO : topic #2 (0.100): 0.005*"poland" + 0.004*"polish" + 0.003*"nochlin" + 0.003*"sexism" + 0.003*"reilly" + 0.002*"destinations" + 0.002*"binet" + 0.002*"irwin" + 0.002*"orlan" + 0.002*"pis"
2021-03-14 20:31:12,506 : INFO : topic #8 (0.100): 0.010*"drahi" + 0.006*"ant" + 0.006*"capsule" + 0.005*"farm" + 0.004*"vermeer" + 0.004*"lst" + 0.004*"artpark" + 0.003*"citizens" + 0.003*"altice" + 0.002*"sfr"
2021-03-14 20:31:12,509 : INFO : topic #9 (0.100): 0.002*"umbrella" + 0.002*"tobacco" + 0.002*"nope" + 0.001*"airshow" + 0.001*"ai

2021-03-14 20:31:15,773 : INFO : topic #4 (0.100): 0.048*"pm" + 0.018*"street" + 0.017*"time" + 0.016*"price" + 0.014*"location" + 0.014*"saturday" + 0.013*"free" + 0.013*"gallery" + 0.009*"newyork" + 0.009*"tuesday"
2021-03-14 20:31:15,777 : INFO : topic diff=0.147674, rho=0.377964
2021-03-14 20:31:15,806 : INFO : PROGRESS: pass 4, at document #100/300
2021-03-14 20:31:16,021 : INFO : merging changes from 100 documents into a model of 300 documents
2021-03-14 20:31:16,052 : INFO : topic #5 (0.100): 0.007*"million" + 0.006*"gallery" + 0.006*"works" + 0.005*"fair" + 0.005*"just" + 0.005*"market" + 0.004*"world" + 0.004*"time" + 0.004*"people" + 0.004*"new"
2021-03-14 20:31:16,057 : INFO : topic #2 (0.100): 0.006*"poland" + 0.005*"polish" + 0.003*"nochlin" + 0.003*"sexism" + 0.003*"reilly" + 0.002*"binet" + 0.002*"destinations" + 0.002*"statistics" + 0.002*"closes" + 0.002*"orlan"
2021-03-14 20:31:16,060 : INFO : topic #4 (0.100): 0.049*"pm" + 0.017*"time" + 0.016*"street" + 0.015*"price

2021-03-14 20:31:19,521 : INFO : topic #3 (0.100): 0.038*"gallery" + 0.029*"galerie" + 0.027*"newyork" + 0.013*"berlin" + 0.011*"london" + 0.010*"paris" + 0.007*"born" + 0.007*"lives" + 0.006*"los" + 0.006*"angeles"
2021-03-14 20:31:19,528 : INFO : topic diff=0.096757, rho=0.333333
2021-03-14 20:31:19,551 : INFO : PROGRESS: pass 6, at document #100/300
2021-03-14 20:31:19,817 : INFO : merging changes from 100 documents into a model of 300 documents
2021-03-14 20:31:19,840 : INFO : topic #4 (0.100): 0.047*"pm" + 0.017*"time" + 0.016*"street" + 0.014*"price" + 0.013*"saturday" + 0.013*"location" + 0.013*"free" + 0.012*"gallery" + 0.009*"tuesday" + 0.009*"newyork"
2021-03-14 20:31:19,843 : INFO : topic #3 (0.100): 0.031*"gallery" + 0.024*"newyork" + 0.024*"galerie" + 0.012*"berlin" + 0.009*"london" + 0.008*"paris" + 0.006*"born" + 0.005*"los" + 0.005*"angeles" + 0.005*"lives"
2021-03-14 20:31:19,845 : INFO : topic #5 (0.100): 0.007*"million" + 0.006*"gallery" + 0.006*"works" + 0.005*"just

2021-03-14 20:31:23,110 : INFO : topic #4 (0.100): 0.045*"pm" + 0.017*"street" + 0.016*"time" + 0.015*"price" + 0.013*"gallery" + 0.013*"location" + 0.013*"saturday" + 0.012*"free" + 0.009*"newyork" + 0.008*"tuesday"
2021-03-14 20:31:23,111 : INFO : topic diff=0.082891, rho=0.301511
2021-03-14 20:31:23,143 : INFO : PROGRESS: pass 8, at document #100/300
2021-03-14 20:31:23,410 : INFO : merging changes from 100 documents into a model of 300 documents
2021-03-14 20:31:23,455 : INFO : topic #7 (0.100): 0.009*"greco" + 0.001*"blake" + 0.001*"feininger" + 0.001*"dweck" + 0.001*"nobuyoshi" + 0.001*"sturges" + 0.001*"mel" + 0.001*"erwin" + 0.001*"araki" + 0.001*"delaunay"
2021-03-14 20:31:23,464 : INFO : topic #5 (0.100): 0.008*"million" + 0.007*"gallery" + 0.005*"just" + 0.005*"works" + 0.005*"fair" + 0.005*"market" + 0.005*"people" + 0.004*"world" + 0.004*"time" + 0.004*"think"
2021-03-14 20:31:23,475 : INFO : topic #3 (0.100): 0.032*"gallery" + 0.024*"newyork" + 0.023*"galerie" + 0.012*"be

...

2021-03-14 20:32:25,693 : INFO : topic #0 (0.100): 0.009*"museum" + 0.005*"new" + 0.004*"works" + 0.004*"world" + 0.004*"time" + 0.004*"exhibition" + 0.003*"newyork" + 0.003*"year" + 0.003*"years" + 0.003*"director"
2021-03-14 20:32:25,695 : INFO : topic diff=0.035458, rho=0.145865
2021-03-14 20:32:25,741 : INFO : PROGRESS: pass 44, at document #100/300
2021-03-14 20:32:25,960 : INFO : merging changes from 100 documents into a model of 300 documents
2021-03-14 20:32:26,028 : INFO : topic #2 (0.100): 0.005*"poland" + 0.004*"polish" + 0.003*"nochlin" + 0.002*"sexism" + 0.002*"reilly" + 0.002*"statistics" + 0.002*"counting" + 0.002*"binet" + 0.002*"destinations" + 0.002*"closes"
2021-03-14 20:32:26,031 : INFO : topic #0 (0.100): 0.009*"museum" + 0.005*"new" + 0.004*"works" + 0.004*"world" + 0.004*"time" + 0.004*"exhibition" + 0.003*"newyork" + 0.003*"year" + 0.003*"years" + 0.002*"director"
2021-03-14 20:32:26,032 : INFO : topic #1 (0.100): 0.010*"moma" + 0.007*"add" + 0.007*"food" + 0.00

2021-03-14 20:32:29,114 : INFO : topic #8 (0.100): 0.012*"vermeer" + 0.007*"drahi" + 0.005*"capsule" + 0.004*"ant" + 0.004*"farm" + 0.004*"girl" + 0.003*"lst" + 0.003*"artpark" + 0.003*"earring" + 0.003*"citizens"
2021-03-14 20:32:29,115 : INFO : topic diff=0.034546, rho=0.142857
2021-03-14 20:32:29,161 : INFO : PROGRESS: pass 46, at document #100/300
2021-03-14 20:32:29,406 : INFO : merging changes from 100 documents into a model of 300 documents
2021-03-14 20:32:29,458 : INFO : topic #7 (0.100): 0.008*"greco" + 0.001*"searches" + 0.001*"feininger" + 0.001*"blake" + 0.001*"grosz" + 0.001*"mel" + 0.001*"araki" + 0.001*"dweck" + 0.001*"nobuyoshi" + 0.001*"sturges"
2021-03-14 20:32:29,465 : INFO : topic #8 (0.100): 0.010*"vermeer" + 0.009*"drahi" + 0.006*"capsule" + 0.005*"ant" + 0.005*"farm" + 0.004*"lst" + 0.004*"artpark" + 0.004*"girl" + 0.004*"citizens" + 0.003*"reading"
2021-03-14 20:32:29,476 : INFO : topic #3 (0.100): 0.034*"gallery" + 0.023*"newyork" + 0.022*"galerie" + 0.015*"be

2021-03-14 20:32:32,848 : INFO : topic diff=0.033929, rho=0.140028
2021-03-14 20:32:32,882 : INFO : PROGRESS: pass 48, at document #100/300
2021-03-14 20:32:33,125 : INFO : merging changes from 100 documents into a model of 300 documents
2021-03-14 20:32:33,163 : INFO : topic #0 (0.100): 0.009*"museum" + 0.005*"new" + 0.004*"works" + 0.004*"world" + 0.004*"time" + 0.004*"exhibition" + 0.003*"newyork" + 0.003*"year" + 0.003*"years" + 0.002*"director"
2021-03-14 20:32:33,166 : INFO : topic #4 (0.100): 0.041*"pm" + 0.016*"time" + 0.015*"street" + 0.013*"price" + 0.013*"gallery" + 0.012*"location" + 0.012*"saturday" + 0.012*"free" + 0.009*"newyork" + 0.008*"tuesday"
2021-03-14 20:32:33,175 : INFO : topic #6 (0.100): 0.002*"pulse" + 0.001*"hendeles" + 0.001*"jenney" + 0.001*"ressle" + 0.001*"donkey" + 0.001*"molyneux" + 0.001*"vieira" + 0.001*"thek" + 0.001*"mestrovic" + 0.001*"nu"
2021-03-14 20:32:33,176 : INFO : topic #2 (0.100): 0.005*"poland" + 0.004*"polish" + 0.003*"nochlin" + 0.002*"

2021-03-14 20:32:36,530 : INFO : topic diff=0.033214, rho=0.137361
2021-03-14 20:32:36,549 : INFO : topic #0 (0.100): 0.009*"museum" + 0.005*"new" + 0.004*"works" + 0.004*"world" + 0.004*"time" + 0.004*"exhibition" + 0.003*"newyork" + 0.003*"year" + 0.003*"years" + 0.003*"director"
2021-03-14 20:32:36,553 : INFO : topic #1 (0.100): 0.008*"moma" + 0.006*"food" + 0.006*"add" + 0.005*"local" + 0.005*"salt" + 0.005*"wearemoma" + 0.004*"iammoma" + 0.004*"shared" + 0.004*"oil" + 0.004*"post"
2021-03-14 20:32:36,559 : INFO : topic #2 (0.100): 0.004*"poland" + 0.003*"polish" + 0.002*"nochlin" + 0.002*"sexism" + 0.002*"reilly" + 0.002*"statistics" + 0.002*"counting" + 0.002*"binet" + 0.002*"destinations" + 0.002*"closes"
2021-03-14 20:32:36,561 : INFO : topic #3 (0.100): 0.036*"gallery" + 0.024*"newyork" + 0.024*"galerie" + 0.016*"berlin" + 0.009*"london" + 0.008*"paris" + 0.007*"born" + 0.007*"lives" + 0.006*"david" + 0.006*"los"
2021-03-14 20:32:36,564 : INFO : topic #4 (0.100): 0.040*"pm" + 

[(0,
  '0.009*"museum" + 0.005*"new" + 0.004*"works" + 0.004*"world" + 0.004*"time" + 0.004*"exhibition" + 0.003*"newyork" + 0.003*"year" + 0.003*"years" + 0.003*"director"'),
 (1,
  '0.008*"moma" + 0.006*"food" + 0.006*"add" + 0.005*"local" + 0.005*"salt" + 0.005*"wearemoma" + 0.004*"iammoma" + 0.004*"shared" + 0.004*"oil" + 0.004*"post"'),
 (2,
  '0.004*"poland" + 0.003*"polish" + 0.002*"nochlin" + 0.002*"sexism" + 0.002*"reilly" + 0.002*"statistics" + 0.002*"counting" + 0.002*"binet" + 0.002*"destinations" + 0.002*"closes"'),
 (3,
  '0.036*"gallery" + 0.024*"newyork" + 0.024*"galerie" + 0.016*"berlin" + 0.009*"london" + 0.008*"paris" + 0.007*"born" + 0.007*"lives" + 0.006*"david" + 0.006*"los"'),
 (4,
  '0.040*"pm" + 0.016*"street" + 0.015*"time" + 0.013*"gallery" + 0.013*"price" + 0.012*"location" + 0.012*"saturday" + 0.011*"free" + 0.009*"newyork" + 0.008*"sarah"'),
 (5,
  '0.008*"million" + 0.008*"gallery" + 0.006*"just" + 0.006*"people" + 0.006*"market" + 0.005*"fair" + 0.005*"w

In [144]:
import gensim.corpora as corpora# Create Dictionary
id2word = corpora.Dictionary(data_clean['para'])

#print(lda.print_topics())
dictlda = lda[corpus]
dictlda

2021-03-14 20:35:00,701 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [116]:
#!/usr/bin/python3 -m pip install pyldavis
import pyLDAvis.gensim
import pickle 
import pyLDAvis# Visualize the topics
import pyLDAvis.sklearn
import gensim
#???
#from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [146]:
#!mkdir results

In [149]:
pyLDAvis.enable_notebook()

num_topics = 10
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))


# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, corpus, id2word)
    #pyLDAvis.sklearn.prepare(lda, corpus, cv)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)# load the pre-prepared pyLDAvis data from disk

#with open(LDAvis_data_filepath, 'rb') as f:
    #LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

AttributeError: 'dict' object has no attribute 'token2id'

In [85]:
print(corpus)